In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten ,Dense ,Dropout ,BatchNormalization
from tensorflow.keras.optimizers import Adamax

from tensorflow.keras.utils import plot_model #for model visualization

In [ ]:
data_with_mask_dir = "/kaggle/input/face-mask-dataset/data/with_mask"
data_without_mask_dir = "/kaggle/input/face-mask-dataset/data/without_mask"

In [ ]:
data_with_mask = os.listdir(data_with_mask_dir)[:5]
data_without_mask = os.listdir(data_without_mask_dir)[:5]

In [ ]:
for img_file in data_with_mask:
    img_path = os.path.join(data_with_mask_dir, img_file)
    img = Image.open(img_path)
    plt.imshow(img)
    plt.title('Image with Mask')
    plt.axis('off')
    plt.show()

In [ ]:
for img_file in data_without_mask:
    img_path = os.path.join(data_without_mask_dir, img_file)
    img = Image.open(img_path)
    plt.imshow(img)
    plt.title("Image without Mask")
    plt.axis('off')
    plt.show()

In [ ]:
data_with_mask = os.listdir(data_with_mask_dir)
data_without_mask = os.listdir(data_without_mask_dir)

In [ ]:
print("Number of with mask image :- ", len(data_with_mask))

In [ ]:
print("Number of without mask image :- ", len(data_without_mask))

In [ ]:
with_mask_labels = [1]*len(data_with_mask)
print(f"with mask lebels", with_mask_labels[0:10])
without_mask_labels = [0]*len(data_without_mask)
print(f"without mask labels", without_mask_labels[0:10])

In [ ]:
labels = with_mask_labels+without_mask_labels
print(f"labels", labels[0:10])
print(f"labels", labels[-10:])

In [ ]:
images = []
def load_images_from_folder(folder):
    for filename in os.listdir(folder):
        img = Image.open(os.path.join(folder,filename))
        img = img.resize((128,128))
        img = img.convert('RGB')
        images.append(np.array(img))
    return images
with_mask_images = load_images_from_folder("/kaggle/input/face-mask-dataset/data/with_mask")
without_mask_images = load_images_from_folder("/kaggle/input/face-mask-dataset/data/without_mask")

In [ ]:
len(images)

In [ ]:
images[0].shape

In [ ]:
x = np.array(images)
y = np.array(labels)

In [ ]:
print(x.shape)

In [ ]:
y.shape

In [ ]:
print(y.shape)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3, random_state=42)

In [ ]:
print(f"shape of x_trian is: {x_train.shape}")
print(f"shape of y_train_is: {y_train.shape}")
print(f"shape of x_test_is: {x_test.shape}")
print(f"shape of y_test_is: {y_test.shape}")
      

In [ ]:
x_train = x_train/255
x_test = x_test/255

In [ ]:
image_size = (128,128)
channels = 3

image_shape = (image_size[0], image_size[1], channels)

base_model = tf.keras.applications.ResNet152V2(
    include_top=False,
    weights='imagenet',
    input_shape=image_shape)
model = Sequential ([
    base_model,
    Flatten(),
    BatchNormalization(),
    Dense(256, activation='relu'),
    Dropout(rate=0.2),
    BatchNormalization(),
    Dense(128, activation ='relu'),
    Dropout(rate=0.2),
    BatchNormalization(),
    Dense(64, activation='relu'),
    Dropout(rate=0.2),
    Dense(2, activation = 'sigmoid')
])

In [ ]:
model.compile(Adamax(learning_rate=0.001),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit( x_train, y_train, epochs=10, validation_split=0.1)

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print('Test Accuracy =', accuracy )

In [ ]:
from sklearn.metrics import classification_report 
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred))

In [ ]:
input_image_path = "/kaggle/input/face-mask-dataset/data/without_mask/without_mask_100.jpg"
input_image = cv2.imread(input_image_path)
plt.imshow(input_image)
plt.show()
input_image_resized = cv2.resize(input_image,(128,128))
input_image_scaled = input_image_resized/255
input_image_reshaped =np.reshape(input_image_scaled, [1,128,128,3])
input_prediction = model.predict(input_image_reshaped)
print(input_prediction)
input_pred_label=np.argmax(input_prediction)
print(input_pred_label)

if input_pred_label == 1:
    print("The person in the image is wearing a mask")
else:
    print("The person in the image is not wearing a mask")


In [ ]:
input_image_path = "/kaggle/input/face-mask-dataset/data/with_mask/with_mask_1002.jpg"
input_image = cv2.imread(input_image_path)
plt.imshow(input_image)
plt.show()
input_image_resized = cv2.resize(input_image,(128,128))

input_image_scaled = input_image_resized/255
input_image_reshaped = np.reshape(input_image_scaled,[1,128,128,3])
input_prediction = model.predict(input_image_reshaped)
print(input_prediction)
input_pred_label = np.argmax(input_prediction)
print(input_pred_label)

if input_pred_label == 1:
    print("The person in the image is wearing a mask")
    
else:
    print("The person in the image in not wearing a mask")
